In [ ]:
import numpy as np
import random
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_mat import *
from ids import *
from block_maxvol import *
from block_rect_maxvol import *
import scipy.linalg
from gen_1D import matrix_prep
%matplotlib inline

### variants to generate matrix $A$

In [ ]:
n = 5000 #nmber of sampling points   
deg = 9 #Degree of polynomial       
interval = [0, 5]  # interval of x
x= np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1
nder = 1

In [ ]:
nder = 2 # dimension
n = 4 # number of points
n_test = 5000 #points on test grid
p_size = (nder+1)*2 #number of monoms (number of columns)
interval = [-2, 2]  # interval of x_test
np.random.seed(0)
x = 2*np.random.rand(n, nder) - 1


In [ ]:
c = GenMat(p_size, x,poly=cheb, debug=False,pow_p=1)
print c.shape
# turn matrix to the block structure
A = matrix_prep(c,nder+1,n)
print A.shape

In [ ]:
max_val = 15
n, p_size = 18, 6
mat_int = np.random.uniform( low=-max_val, high=max_val, size=n*p_size).reshape((n,p_size)).astype(int)
A = mat_int.astype(float)
nder = 2

In [ ]:
A = np.array([[-8.+1j, -7.,  1.,  0. ,-5.,  2.],
 [ 6., -1., -8.,  4.,  4., -9.],
 [ 9.,  5.,  8.,  0., -6., -4.],
 [ 6.,  5., -6.,  2., -9.,  8.],
 [ 3., -8.,  3., -3.,  4., -4.],
 [-7. , 2.,  0., -6.,  5., -5.],
 [ 6.  ,1., -1.,  0., -2.,  1.],
 [ 1.  ,9., -6., -4.,  0.,  0.],
 [-2.  ,0.,  0., -7., -6.,  4.],
 [-1.  ,0., -1.,  9., -4.,  4.],
 [ 4., -4. , 1.,  7., -6., -9.],
 [-7.,  8.,  4.,  1.,  0.,  0.]])
print A
nder = 3
p_size = A.shape[1]

### start information about loaded matrix

In [ ]:
print la.det(A[:A.shape[1]])
print la.matrix_rank(A)
print la.matrix_rank(A[:A.shape[1]])
_,s,_ = scipy.linalg.svd(A)
print s

piv,_ = maxvol(A)
print piv
print la.det(A[piv])

###  test PLUQ Preprocessing separately

In [ ]:
def plu_index(A):
    n, m = A.shape[0], A.shape[1]
    P = np.eye((n), dtype=float)
    L = np.eye(n, m, dtype=A.dtype)
    U = np.copy(A)
    permP = np.arange(n)
    Urow = np.arange(n)
    Ucol = np.arange(m)
    for j in range(0, m):
        loc_max = np.argmax(np.abs(U[Urow[j:]][Ucol[j]]))
        print np.abs(U[Urow[j:],Ucol[j]])
        form_permute(Urow,j,loc_max)
        print (j,loc_max)
        L[[j+loc_max,j],:j] = L[[j,j+loc_max], :j]
        
        form_permute(permP,j,loc_max + j)
        
        
        for i in range(j+1, n):
            L[i,j] = U[Urow[i],Ucol[j]]/U[Urow[j],Ucol[j]]
            U[Urow[i],Ucol[j:]] -= L[i,j]*U[Urow[j],Ucol[j:]]
           
    return(permP, L, U[Urow][:,Ucol])  

In [ ]:
def plu(A, Tru=True):
    n, m = A.shape[0], A.shape[1]
    P = np.eye((n), dtype=float)
    L = np.eye(n, m, dtype=A.dtype)
    U = np.copy(A)
    for j in range(0, m):
        loc_max = np.argmax(np.abs(U[j:, j]))

        U[[j+loc_max,j],j:] = U[[j,j+loc_max], j:]
        
        if Tru:
            L[[j+loc_max,j],:j] = L[[j,j+loc_max], :j]
        else:
            L[[j+loc_max,j],:] = L[[j,j+loc_max], :]
        
        P[[j+loc_max,j],:] = P[[j,j+loc_max], :]
        
        
        for i in range(j+1, n):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:] -= L[i,j]*U[j,j:]
           
    return(P, L, U) 

In [ ]:
A = np.random.randn(5,5)
print np.linalg.matrix_rank(A)
print A

In [ ]:
P1, L1, U1 = plu(A, True)

In [ ]:
P2, L2, U2 = plu(A, False)

In [ ]:
print P1==P2
print L1==L2
print U1==U2

In [ ]:
np.max(np.abs(  (P1.dot(A) - (L1.dot(U1))).flatten()))
np.max(np.abs(  (P2.dot(A) - (L2.dot(U2))).flatten()))

In [ ]:
for i in range(800):
    dim = random.randint(1, 8)
    print ('progress= ', i, 'dimension= ', dim)
    Ncol = dim*random.randint(1,20)
    Nrow = random.randint(2,20)*Ncol
    A = np.random.randn(Nrow,Ncol)
    p,l,u,q,inf = pluq_ids_index(A,dim-1,debug=False)
    p1,l1,u1,q1,inf1 = pluq_ids(A,dim-1,debug=False)
    if not np.all(p==p1):
        print A
        print (p,p1)

In [ ]:
#p,l,u,q,indx = pluq_ids_index(A,nder)
try:
    p,l,u,q,indx = pluq_ids_index(A,nder)
    print la.matrix_rank((perm_matrix(p).dot(A))[:6])
except SingularError:
    print 'not full column rank'

### test Block maxvol perfomance separately

In [ ]:
%time r,t,y = block_maxvol(A_init,nder, tol=0.0,max_iters=200,swm_upd=True)

In [ ]:
%time C, B, P = block_maxvol(A_init, nder,tol=0.0,max_iters = 2000,swm_upd=False)

In [ ]:
print np.linalg.det((b[:b.shape[1]]))
print np.linalg.det((A_init[:b.shape[1]]))
print np.linalg.det((r[:b.shape[1]]))
print np.linalg.det((C[:b.shape[1]]))

In [ ]:
print y[:b.shape[1]] # swm permuts
print P[:b.shape[1]] # recalc A^{-1} permuts
print pr[:b.shape[1]]

### test rect_block_maxvol

In [ ]:
k = 8
assert k//(nder+1)!=0
assert (k>=A.shape[1])
a,b,c,d,e = rect_block_maxvol(A, nder, Kmax = k, max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, ext_debug = True)

In [ ]:
e[:k]

In [ ]:
d[:k]

In [ ]:
c[:k]

In [ ]:
print A[d]

In [ ]:
A_test = A[d]
C_test = np.dot(A_test, la.pinv(A_test[:4]))
ndim = nder+1
m = C_test.shape[0] // ndim
n = C_test.shape[0]
values = []
for i in range(0,m):
    CC_T = np.dot(C_test[i*ndim:i*ndim+ndim], C_test[i*ndim:i*ndim+ndim].T)
    values.append((CC_T))
ind = la.det(np.eye(ndim) + values)

In [ ]:
print ind.reshape((5,1))

In [ ]:
print A[c]

In [ ]:
A_test = A[c]

In [ ]:
C_test = np.dot(A_test, la.pinv(A_test[:k]))

In [ ]:
la.det(np.dot(A_test[:6].T,A_test[:6]))

In [ ]:
A_test = A[d]
print(la.det(np.dot(A_test[:6].T,A_test[:6])))

In [ ]:
np.linalg.norm((C_test - a))

In [ ]:
### returns 2 values - function on domain, and block structured
def rhs(points, nder, mode = 'gauss'):
    if mode == 'gauss':
        block_rhs = np.zeros((nder+1)*(points.shape[0]))
        func = 2*np.exp(-((points[:,0]**2)/2. + (points[:,1]**2)/2.))# + (points[:,2]**2)/2.)) # Gaussian
        for i in range(points.shape[0]):
            block_rhs[i*(nder+1)] = func[i]
            for j in range(nder):
                block_rhs[i*(nder+1)+j+1] = -1*(points[i,j])*func[i]
    if mode == '3D' :
        block_rhs = np.zeros((nder+1)*(points.shape[0]))
        func = 2*((points[:,0]**2)/2. + (points[:,1]**2)/2. + (points[:,2]**2)/2.) # Gaussian
        for i in range(points.shape[0]):
            block_rhs[i*(nder+1)] = func[i]
            for j in range(nder):
                block_rhs[i*(nder+1)+j+1] = 2 * points[i,j]
    return func, block_rhs     

In [ ]:
### solve LSM
f,t = rhs(x,nder, mode = 'gauss')
A_test = GenMat(p_size, points_test,poly=cheb, debug=False,pow_p=1, ToGenDiff=False)
f_test,_ = rhs(points_test,nder)

### Take number of rows that we really need
c = final_piv[:k]

c_block, res_x, rank, s = np.linalg.lstsq(A[c],t[c])

taken_p = x[c[::(nder+1)]/(nder+1),:]

# visualization part
# position of the chosen points by MaxVol and GD in 2-D case
l_bound = interval[0]
u_bound = interval[1]
plt.xlim(l_bound-0.15, u_bound+0.15)
plt.ylim(l_bound-0.15, u_bound+0.15)
#plt.plot(M[row_indx,1], M[row_indx,2], 'ro', label = "MV")
plt.plot(taken_p[:,0],taken_p[:,1], 'b^', label = "BMV")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, borderaxespad=0.)
plt.grid(True)
plt.show()

print("GD error = ", la.norm(f_test - np.dot(A_test, c_block), np.inf) / la.norm(f_test, np.inf), "\n")